In [1]:
# | default_exp train.nbeats

In [2]:
import torch
import wandb
from pytorch_lightning.profilers import PyTorchProfiler

In [4]:
import pandas as pd
from datasetsforecast.m3 import M3
from datasetsforecast.m4 import M4

# | export
from datasetsforecast.m5 import M5

df = pd.concat(
    [
        M3().load("../data", group="Monthly")[0],
        M4().load("../data", group="Monthly")[0],
        M5().load("../data")[0],
    ]
)

df.sort_values(["unique_id", "ds"], inplace=True)
df.head()

/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packages/datasetsforecast/m3.py:108: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  freq = pd.tseries.frequencies.to_offset(class_group.freq)


ModuleNotFoundError: No module named 'numpy._core.numeric'

In [3]:
df.unique_id.nunique()

30490

In [4]:
df.groupby("unique_id").apply(len).describe()

/tmp/ipykernel_98679/840638693.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby("unique_id").apply(len).describe()
/tmp/ipykernel_98679/840638693.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby("unique_id").apply(len).describe()


count    30490.000000
mean      1562.805510
std        477.176658
min        124.000000
25%       1203.000000
50%       1810.000000
75%       1968.000000
max       1969.000000
dtype: float64

In [5]:
from ts.model.nbeats import NBeatsG

horizon = 12
input_size = horizon * 5

model = NBeatsG(input_size=input_size, horizon=horizon)

In [6]:
# | export
from ts.preprocess.dataloader import UnivariateTSDataModule

In [7]:
# | export

batch_size = 512 * 10
num_workers = 24
step_size = 6

ds = UnivariateTSDataModule(
    df=df,
    input_size=input_size,
    horizon=horizon,
    batch_size=batch_size,
    num_workers=num_workers,
    train_split=0.7,
    val_split=0.15,
    normalize=True,
    scaler_type="minmax",
    split_type="vertical",
    step_size=step_size,
    prefetch_factor=4,
)

In [8]:
# | export
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

In [9]:
# | export
# Example trainer setup (without full NBeatsG for brevity)

wandb_logger = WandbLogger(
    project="benchmark-ts-global-forecast",
    name=f"model=NBeatsG.ds=M5",
)
wandb_logger.watch(model, log="all")

profiler = PyTorchProfiler(
    profile_memory=True,  # Track GPU memory
    record_shapes=True,
    with_stack=True,  # Track CPU memory (if supported)
)

trainer = pl.Trainer(
    logger=wandb_logger,
    max_epochs=200,  # Short run for testing
    accelerator="auto",
    logger=TensorBoardLogger("logs", name="nbeatsg_m5"),
    callbacks=[EarlyStopping("val_smape", patience=10, verbose=False)],
    # profiler=profiler,
    accumulate_grad_batches=4,
    # strategy="ddp_notebook"
)


trainer.fit(model, ds)

INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/home/pranav-pc/projects/ts/ts/preprocess/dataloader.py:274: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = self.df.groupby("unique_id")
INFO:ts.preprocess.dataloader:Train windows: 6765731, Val windows: 1449641, Test windows: 1449958
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/pranav-pc/projects/ts/.venv/lib/python3.12/site-packag

Sanity Checking: |                                                                                            …

Training: |                                                                                                   …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

...

Validation: |                                                                                                 …

Validation: |                                                                                                 …

Validation: |                                                                                                 …

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
24 * 4

In [ ]:
# | export
trainer.test(model, ds);

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torch
from plotly.subplots import make_subplots


def forecast_and_plot_grid(trainer, model, data_module, num_series=6):
    # Ensure model is in evaluation mode
    model.eval()

    # Get all unique_ids from the DataFrame
    unique_ids = data_module.df["unique_id"].unique()
    # Randomly select 6 unique_ids (or fewer if less available)
    selected_ids = np.random.choice(
        unique_ids, size=min(num_series, len(unique_ids)), replace=False
    )

    # Prepare test data for selected series
    test_data = data_module.df[data_module.df["unique_id"].isin(selected_ids)]
    grouped = test_data.groupby("unique_id")

    # Create figure with 3x2 grid
    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=[f"Series: {uid}" for uid in selected_ids],
        vertical_spacing=0.15,
        horizontal_spacing=0.1,
    )

    # Perform forecasting and plotting
    for i, unique_id in enumerate(selected_ids):
        series_df = grouped.get_group(unique_id)
        series = series_df["y"].values
        series_len = len(series)

        if series_len < data_module.input_size + data_module.horizon:
            print(f"Skipping {unique_id}: too short for forecasting")
            continue

        # Normalize series if data_module uses normalization
        if data_module.normalize:
            scaler = data_module.scalers.get(unique_id)
            if scaler:
                series_normalized = scaler.transform(series.reshape(-1, 1)).flatten()
            else:
                series_normalized = series
        else:
            series_normalized = series

        # Generate last input window for forecasting
        last_input = series_normalized[
            -data_module.input_size - data_module.horizon : -data_module.horizon
        ]
        x = torch.tensor(last_input, dtype=torch.float32).unsqueeze(0).to(model.device)

        # Forecast
        with torch.no_grad():
            y_hat = model(x).cpu().numpy().flatten()

        # Inverse transform predictions and actual values
        if data_module.normalize and scaler:
            y_hat_denorm = scaler.inverse_transform(y_hat.reshape(-1, 1)).flatten()
            series_denorm = series  # Already in original scale
        else:
            y_hat_denorm = y_hat
            series_denorm = series

        # Time indices for the entire series and forecast
        full_time_indices = series_df["ds"].values  # Full series dates
        forecast_time_indices = full_time_indices[-data_module.horizon :]  # Last horizon dates

        # Determine row and column (0-based indexing, converting to 1-based for Plotly)
        row = (i // 2) + 1
        col = (i % 2) + 1

        # Plot the entire actual series
        fig.add_trace(
            go.Scatter(
                x=full_time_indices,
                y=series_denorm,
                mode="lines+markers",
                line=dict(color="blue"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

        # Plot predicted values for the last horizon
        fig.add_trace(
            go.Scatter(
                x=forecast_time_indices,
                y=y_hat_denorm,
                mode="lines+markers",
                line=dict(color="red", dash="dash"),
                showlegend=False,
            ),
            row=row,
            col=col,
        )

    # Update layout
    fig.update_layout(
        height=900,
        width=800,  # Fixed size for 3x2 grid
        title_text="N-BEATS-G Forecasting: Full Series with Last Horizon Predicted (3x2 Grid)",
        showlegend=False,  # Remove legend entirely
    )
    fig.update_yaxes(title_text="Value")
    fig.update_xaxes(title_text="Date")

    # Show plot
    fig.show()


# Example usage (assuming trainer, model, and data_module are defined)
# Replace 'ds' with 'data_module' if that’s your variable name
forecast_and_plot_grid(trainer, model, ds, num_series=6)

In [ ]:
# | export
trainer.save_checkpoint("M5-profile.ckpt")